In [1]:
data1 = [(7369, "SMITH", "CLERK", 7902, "17-12-1980", 800, None, 20),
        (7499, "ALLEN", "SALESMAN",7698,"20-02-1981",1600,300,  30),
        (7521, "WARD", "SALESMAN", 7698,"22-02-1981",1250,500,  30),
        (7566, "JONES", "MANAGER", 7839, "02-04-1981",2975,None,20),
        (7654, "MARTIN", "SALESMAN", 7698,"28-09-1981",1250,1400,30),
        (7698, "BLAKE", "MANAGER", 7839, "01-05-1981", 2850, None,30),
        (7782, "CLARK", "MANAGER", 7839, "09-06-1981", 2450, None, 10),
        (7788, "SCOTT", "ANALYST", 7566, "09-12-1982", 3000, None, 20),
        (7839, "KING", "PRESIDENT",None, "17-11-1981", 5000, None, 10),
        (7844, "Turner", "SALESMAN",7698, "08-09-1981", 1500, 0,  30),
        (7876, "ADAMS", "CLERK",   7788,  "12-01-1983", 1100, None, 20),
        (7900, "JAMES",  "CLERK",  7698,  "03-12-1981", 950, None, 30),
        (7902, "FORD",  "ANALYST", 7566,  "03-12-1981", 3000, None, 20),
        (7934, "MILLER", "CLERK",  7782,  "23-01-1982", 1300, None, 10)
       ]
schema1 = ["empno","ename","job","mgr","hiredate","sal","comm","deptno"]

In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
emp_df = spark.createDataFrame(data1,schema1)
emp_df.show() 
emp_df.printSchema()

25/02/27 10:01:26 WARN Utils: Your hostname, user-HP-Pavilion-x360-Convertible-14-dh0xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlo1)
25/02/27 10:01:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 10:01:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-12-1980| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|20-02-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-02-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|02-04-1981|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|28-09-1981|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|01-05-1981|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|09-06-1981|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|09-12-1982|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|
| 7844|Turner| SALESMAN|7698|08-09-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|12-01-1983|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|03-12-1981| 950|NULL|    30|
| 7902|  FORD|  ANALYST|7566|03-12-1981|3000|NULL|    20|
| 7934|MILLER|    CLERK|7782|23-01-1982|1300|NULL|    10|
+-----+------+

In [3]:
emp_df.createOrReplaceTempView('emp')

In [4]:
#emp_df = emp_df1.select(col("hiredate"),to_date(col("hiredate"),"dd-MM-yyyy").alias("to_date"))
#emp_df.show()

# Determining the Number of Days Between Two Dates 

In [4]:
spark.sql('''select datediff(ward_hd, allen_hd)
              from
             (select hiredate as ward_hd from emp
                where ename='WARD') x,
             (select hiredate as allen_hd from emp
                where ename='ALLEN') y
           ''').show()

[Stage 4:====================================>                     (7 + 4) / 11]

+---------------------------+
|datediff(ward_hd, allen_hd)|
+---------------------------+
|                       NULL|
+---------------------------+



In [18]:
allen=emp_df.where(col("ename")=="ALLEN").select(to_date(col("hiredate"),"dd-MM-yyyy")).collect()[0][0]
ward =emp_df.where(col("ename")=="WARD").select(to_date(col("hiredate"),"dd-MM-yyyy")).collect()[0][0] 
print(f"{allen}")
data = [(allen,ward)]; schema = ['start','end'] 
df = spark.createDataFrame(data=data, schema=schema) 
df.show()
df.withColumn("datediff",datediff(col("end"),col("start"))).show()

1981-02-20
+----------+----------+
|     start|       end|
+----------+----------+
|1981-02-20|1981-02-22|
+----------+----------+

+----------+----------+--------+
|     start|       end|datediff|
+----------+----------+--------+
|1981-02-20|1981-02-22|       2|
+----------+----------+--------+



# Determining the Number of Business Days Between Two Dates

In [ ]:
spark.sql(''' 
           select max(case when ename = 'BLAKE'
                 then hiredate end) as blake_hd,
                 max(case when ename = 'JONES'
                 then hiredate end) as jones_hd
           from emp 
           where ename in ('BLAKE','JONES')
          ''').show()

In [ ]:
spark.sql('''select min(hiredate) AS min_hd,
                  max(hiredate) AS max_hd
                  from emp''').show()

In [ ]:
min_hd = emp_df.agg({"to_date":"min"}).collect()[0][0]
print(f"min_hd {min_hd}")
max_hd = emp_df.agg({"to_date":"max"}).collect()[0][0]
print(f"max_hd {max_hd}")
data = [(min_hd,max_hd),]

schema = ["min_hd","max_hd"] 
df_minmax=spark.createDataFrame(data=data, schema=schema) 
df_minmax.show()
df_minmax.withColumn("diff_days", datediff(col("max_hd") , col("min_hd"))).show()

In [ ]:
data = [(min_hd, max_hd),] 
schema = ["start2","end2"] 
df_date = spark.createDataFrame(data=data, schema=schema) 
df_date.show()
df_date = df_date.withColumn("start_year",year(col("start2"))).withColumn("end_year",year(col("end2")))
diff = df_date.withColumn("diff",col("end_year") - col("start_year")).select('diff').collect()[0][0]
print(f'diff={diff}')
df_date = df_date.withColumn("start_month",month(col("start2"))).withColumn("end_month",month(col("end2")))
diff_month = df_date.withColumn("diff_month",col("end_month") - col("start_month")).select("diff_month").collect()[0][0]
#print(f'month {diff_month}')
number_of_months = 3*12 + (-11) 
number_of_years  = number_of_months // 12
print(f'Months:{number_of_months} Years : {number_of_years}')
#df.withColumn("end_year",year("end2")).show()

# Determining the number of Months or Years Between Two Dates

In [ ]:
val min_value = df.agg(min("salary")).head().get(0)
val max_value = df.agg(max("salary")).head().get(0)

# Determining the Date Difference Between the Current Record and the Next Record

In [20]:
from pyspark.sql.window import Window
windowspec = Window.orderBy("hiredate")
df=emp_df.where(emp_df.deptno==10)
df=df.withColumn("to_date",to_date(col("hiredate"),"dd-MM-yyyy"))
df.show() 
date_diff=df.withColumn("next_hiredate",lead("to_date",1).over(windowspec)).where("next_hiredate is not NULL")
#date_diff.show()
date_diff.withColumn("noDays",datediff(col("next_hiredate"),col("to_date"))).show()

+-----+------+---------+----+----------+----+----+------+----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|   to_date|
+-----+------+---------+----+----------+----+----+------+----------+
| 7782| CLARK|  MANAGER|7839|09-06-1981|2450|NULL|    10|1981-06-09|
| 7839|  KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|1981-11-17|
| 7934|MILLER|    CLERK|7782|23-01-1982|1300|NULL|    10|1982-01-23|
+-----+------+---------+----+----------+----+----+------+----------+



25/02/27 11:21:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/27 11:21:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/27 11:21:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+---------+----+----------+----+----+------+----------+-------------+------+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno|   to_date|next_hiredate|noDays|
+-----+-----+---------+----+----------+----+----+------+----------+-------------+------+
| 7782|CLARK|  MANAGER|7839|09-06-1981|2450|NULL|    10|1981-06-09|   1981-11-17|   161|
| 7839| KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|1981-11-17|   1982-01-23|    67|
+-----+-----+---------+----+----------+----+----+------+----------+-------------+------+



25/02/27 11:21:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/27 11:21:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
